In [1]:
import geocoder
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
# extract html data
r = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
html_doc = r.content.decode('utf-8')
soup = BeautifulSoup(html_doc, 'html.parser')
table_tag = soup.find('table', {'class': 'wikitable sortable'})

In [3]:
# extract table header from html
columns = ['PostalCode', 'Borough', 'Neighborhood']

# extract rows from html
row_tags = table_tag.find_all('tr')
rows = []
for row_tag in row_tags:
    item_tags = row_tag.find_all('td')
    if len(item_tags) != len(columns):
        continue
    row = list(map(lambda x: x.text.strip(), item_tags))
    # ignore rows with unassigned borough
    if row[1] == 'Not assigned':
        continue
    # set neighborhood as borough if unassigned
    if row[2] == 'Not assigned':
        row[2] = row[1]
    rows.append(row)

# Requirements

* The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood.
* Only process the cells that have an assigned borough. Ignore cells with a borough that is **Not assigned**.
* More than one neighborhood can exist in one postal code area. These rows will be combined into one row with the neighborhoods separated with a comma.
* If a cell has a borough but a **Not assigned** neighborhood, then the neighborhood will be the same as the borough.
* Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
* In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [4]:
# generate dataframe as requested
df = pd.DataFrame(rows, columns=columns)
df_postal = df.groupby(['PostalCode', 'Borough']).agg(lambda x: ', '.join(set(x))).reset_index()
df_postal

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Morningside, Guildwood, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Ionview, East Birchmount Park, Kennedy Park"
7,M1L,Scarborough,"Oakridge, Golden Mile, Clairlea"
8,M1M,Scarborough,"Cliffcrest, Scarborough Village West, Cliffside"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [5]:
df_postal.shape

(103, 3)

After trying the geocoder package, the location coordinates turned out to be unavailable for me.

So I had to use the csv file instead.

In [6]:
!wget -O Geospatial_Coordinates.csv -q https://cocl.us/Geospatial_data

In [7]:
df_coord = pd.read_csv('Geospatial_Coordinates.csv')
df_coord.columns.values[0] = 'PostalCode'
df_coord

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [8]:
key_column = 'PostalCode'
df_total = df_postal.set_index(key_column).join(df_coord.set_index(key_column), how='left').reset_index()
df_total

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Morningside, Guildwood, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Ionview, East Birchmount Park, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Oakridge, Golden Mile, Clairlea",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Scarborough Village West, Cliffside",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [9]:
df_total.shape

(103, 5)